In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import json
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
annotations_path = '/kaggle/input/kranoknv/Annotations'
video_path = '/kaggle/input/kranoknv/Videos'

# Eda

In [ ]:

# Function to load annotations
def load_annotations(annotations_path):
    annotations = []
    for file_name in os.listdir(annotations_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(annotations_path, file_name)
            with open(file_path, 'r') as f:
                data = json.load(f)
                for frame, frame_data in data.items():
                    frame_number = int(frame.split('_')[-1])  # 'Frame_0000046' -> 46
                    number_of_people = frame_data['numberOfPeople']
                    for pedestrian_data in frame_data['pedestriansData']:
                        try:
                            bbox = list(map(int, pedestrian_data[:4]))  # Convert bbox coordinates to integers
                            behavior = pedestrian_data[4]
                        except KeyError as e:
                            print(f"Missing key {e} in annotation for frame {frame_number} in file {file_name}")
                            continue

                        annotations.append({
                            'video': file_name,
                            'frame': frame_number,
                            'num_pedestrians': number_of_people,
                            'bbox': bbox,
                            'behavior': behavior
                        })
    return pd.DataFrame(annotations)

# Load annotations into a DataFrame
annotations_df = load_annotations(annotations_path)

# Display the first few rows of the DataFrame
print(annotations_df.head())


* **video**: Videonun adı. Örneğin, Normal_00409.json dosyasının adı bu videonun "Normal" olarak etiketlendiğini ve belirli bir numara (00409) ile tanımlandığını gösterir.
* **frame**: Çerçeve numarası. Bu, videodaki hangi kareye ait olduğunun numarasını gösterir.
* **num_pedestrians**: O karede bulunan pedestrian (yaya) sayısı. Örnekte her zaman 1 değeri var, yani her karede sadece bir yaya var.
* **bbox**: Bounding box koordinatları. Bu, pedestrian'ın görüntüde kapladığı alanı gösterir ve [x_min, y_min, x_max, y_max] formatında verilmiştir. Bu değerler, pedestrian'ın sol üst ve sağ alt köşe koordinatlarını belirtir.
* **behavior**: Yaya davranışı. Bu örnekte her zaman Normal olarak belirtilmiş, yani bu karelerde görülen pedestrian normal davranış sergiliyor.

In [ ]:
annotations_df.info()

In [ ]:
behavior_counts = annotations_df['behavior'].value_counts()
print(behavior_counts)

plt.figure(figsize=(10, 6))
sns.countplot(data=annotations_df, x='behavior', order=behavior_counts.index)
plt.title('Davranış Türlerinin Dağılımı')
plt.xlabel('Davranış Türü')
plt.ylabel('Sayı')
plt.show()

In [ ]:
# pedestrians number for each frame
plt.figure(figsize=(12, 6))
sns.histplot(annotations_df['num_pedestrians'], bins=50, kde=True)
plt.title('Her Çerçevedeki Yayaların Sayısı')
plt.xlabel('Yaya Sayısı')
plt.ylabel('Frekans')
plt.show()

In [ ]:
annotations_df['bbox_width'] = annotations_df['bbox'].apply(lambda x: int(x[2]) - int(x[0]))
annotations_df['bbox_height'] = annotations_df['bbox'].apply(lambda x: int(x[3]) - int(x[1]))

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.histplot(annotations_df['bbox_width'], bins=50, kde=True)
plt.title('BBox Genişliği Dağılımı')
plt.xlabel('Genişlik')
plt.ylabel('Frekans')

plt.subplot(1, 2, 2)
sns.histplot(annotations_df['bbox_height'], bins=50, kde=True)
plt.title('BBox Yüksekliği Dağılımı')
plt.xlabel('Yükseklik')
plt.ylabel('Frekans')

plt.tight_layout()
plt.show()

In [ ]:
print(annotations_df.describe())

In [ ]:
sns.pairplot(annotations_df[['frame', 'num_pedestrians', 'bbox_width', 'bbox_height']])
plt.show()

In [ ]:
def extract_bbox_data(annotations_df):
    bbox_data = []
    labels = []
    
    for index, row in annotations_df.iterrows():
        bbox = row['bbox']
        label = row['behavior']
        bbox_data.append(bbox)
        labels.append(label)
    
    return np.array(bbox_data), np.array(labels)

bbox_data, labels = extract_bbox_data(annotations_df)

# Etiketleri sayısal değerlere dönüştürme
label_map = {'Normal': 0, 'Violent': 1}
labels = np.array([label_map[label] for label in labels])

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(bbox_data, labels, test_size=0.2, random_state=42)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Modeli oluşturma
model = Sequential()
model.add(Dense(64, input_shape=(4,), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Modeli derleme
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
def extract_frames(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    os.makedirs(output_dir, exist_ok=True)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_dir, f'frame_{frame_count:04d}.jpg')
        cv2.imwrite(frame_path, frame)
        frame_count += 1
    
    cap.release()
    return frame_count

def predict_behavior(model, bbox_data):
    prediction = model.predict(np.array([bbox_data]))[0]
    return 'Violent' if np.argmax(prediction) == 1 else 'Normal'

new_video_dir = '/kaggle/input/kranoknv/Videos'
new_frames_dir = '/kaggle/working/new_frames'

for new_video_file in os.listdir(new_video_dir):
    new_video_path = os.path.join(new_video_dir, new_video_file)
    new_frame_count = extract_frames(new_video_path, new_frames_dir)
    
    for frame_num in range(new_frame_count):
        frame_path = os.path.join(new_frames_dir, f'frame_{frame_num:04d}.jpg')
        # Karedeki bounding box verilerini DataFrame'den yükleyebilirsiniz.
        frame_annotations = annotations_df[annotations_df['frame'] == frame_num]
        for _, row in frame_annotations.iterrows():
            bbox_data = row['bbox']
            behavior = predict_behavior(model, bbox_data)
             print(f"Frame {frame_num}:Behavior{behavior})